In [2]:
from pathlib import Path

import polars as pl

from stocksense.database import DatabaseHandler

FIXTURE_PATH = Path("../tests/fixtures")

db = DatabaseHandler()

2025-02-23 18:05:08.405 | SUCCESS  | stocksense.database.schema:create_tables:121 - Tables created successfully


In [4]:
a = db.fetch_market_data("BMC")
a

tic,date,close,adj_close,volume
str,date,f64,f64,i64
"""BMC""",2012-06-18,2575.0,2575.0,0
"""BMC""",2012-06-19,2575.0,2575.0,8800
"""BMC""",2012-06-20,2575.0,2575.0,2290
"""BMC""",2012-06-21,2575.0,2575.0,800
"""BMC""",2012-06-22,2575.0,2575.0,800
…,…,…,…,…
"""BMC""",2022-02-24,1610.0,1610.0,0
"""BMC""",2022-02-25,1610.0,1610.0,0
"""BMC""",2022-02-28,1610.0,1610.0,0


In [41]:
# fetch all required data
df = db.fetch_financial_data()
info = db.fetch_stock()
market_df = db.fetch_market_data().sort(["tic", "date"])
insider_df = db.fetch_insider_data()
index_data = db.fetch_index_data()
vix_data = db.fetch_vix_data()

firms = ["AAPL", "COST", "DISH"]
date_interval = (pl.lit("2014-01-01").str.to_date(), pl.lit("2024-12-01").str.to_date())

# filter data
info = info.filter((pl.col("tic").is_in(firms)))
index_data = index_data.filter(
    (pl.col("date") >= date_interval[0]) & (pl.col("date") <= date_interval[1])
)
vix_data = vix_data.filter(
    (pl.col("date") >= date_interval[0]) & (pl.col("date") <= date_interval[1])
)
df = df.filter(
    (pl.col("rdq") >= date_interval[0])
    & (pl.col("rdq") <= date_interval[1])
    & (pl.col("tic").is_in(firms))
)
market_df = market_df.filter(
    (pl.col("date") >= date_interval[0])
    & (pl.col("date") <= date_interval[1])
    & (pl.col("tic").is_in(firms))
)
insider_df = insider_df.filter(
    (pl.col("filling_date") >= date_interval[0])
    & (pl.col("filling_date") <= date_interval[1])
    & (pl.col("tic").is_in(firms))
)

info.write_parquet(FIXTURE_PATH / "stock_data.parquet")
df.write_parquet(FIXTURE_PATH / "financial_data.parquet")
market_df.write_parquet(FIXTURE_PATH / "market_data.parquet")
insider_df.write_parquet(FIXTURE_PATH / "insider_data.parquet")
index_data.write_parquet(FIXTURE_PATH / "index_data.parquet")
vix_data.write_parquet(FIXTURE_PATH / "vix_data.parquet")

In [40]:
df = pl.read_parquet(FIXTURE_PATH / "processed_data.parquet")
df

tdq,tic,datadate,rdq,saleq,cogsq,xsgaq,niq,ebitdaq,cshoq,actq,atq,cheq,rectq,invtq,ppentq,lctq,dlttq,ltq,req,seqq,oancfq,ivncfq,fincfq,dvq,capxq,icaptq,surprise_pct,stock_split,n_purch,val_purch,n_sales,val_sales,insider_balance,roa,ni_ttm,roi,…,size_yoy,f_score,fwd_vol_yoy,fwd_downside_vol_yoy,risk_return_4Q,sortino_4Q,excess_return_4Q,sharpe_ratio_4Q,aggressive_hit,moderate_hit,relaxed_hit,mkt_cap_rank,sector_consumer_staples,sector_information_technology,pe_sec,momentum_sec_yoy,momentum_sec_qoq,size_sec,size_factor,mkt_rel_sec,roa_sec,pe_mkt_rank,pe_sec_rank,ev_ebitda_mkt_rank,ev_ebitda_sec_rank,saleq_yoy_mkt_rank,saleq_yoy_sec_rank,roa_mkt_rank,roa_sec_rank,fcf_ttm_mkt_rank,fcf_ttm_sec_rank,der_mkt_rank,der_sec_rank,price_mom_mkt_rank,price_mom_sec_rank,price_yoy_mkt_rank,price_yoy_sec_rank
date,str,date,date,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i8,u32,f64,u32,f64,f64,f64,f64,f64,…,f64,i8,f64,f64,f64,f64,f64,f64,i8,i8,i8,u32,i8,i8,f64,f64,f64,f64,i8,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2014-03-01,"""AAPL""",2013-12-31,2014-01-27,57594.0,33604.0,4383.0,13072.0,19607.0,24988.516,80347.0,225184.0,40711.0,25198.0,2122.0,15488.0,53769.0,16961.0,95500.0,109125.0,129684.0,22670.0,-15103.0,-7749.0,2769.0,1985.0,146645.0,0.031,0,0,0.0,3,4.994,4.994,null,null,8.914044,…,null,null,0.013801,0.01227,3.66396,4.121218,57.750674,2.635973,1,1,1,1,0,1,null,null,null,469641.334609,0,100.0,null,null,null,null,null,null,null,null,null,null,null,10.0,10.0,10.0,10.0,null,null
2014-06-01,"""AAPL""",2014-03-31,2014-04-23,45646.0,25812.0,4354.0,10223.0,15480.0,24128.86,70541.0,205989.0,41350.0,15820.0,1829.0,15120.0,43208.0,16962.0,85810.0,98683.0,120179.0,13538.0,11741.0,-20407.0,2661.0,1382.0,137141.0,0.14,0,0,0.0,5,4.223,4.223,null,null,7.454372,…,null,null,0.013728,0.013236,2.254985,2.338806,35.078527,1.609697,1,1,1,1,0,1,null,null,0.191552,545484.598149,0,100.0,null,null,null,null,null,null,null,null,null,null,null,5.0,10.0,10.0,10.0,null,null
2014-09-01,"""AAPL""",2014-06-30,2014-07-22,37432.0,20751.0,4453.0,7748.0,12228.0,23956.684,67949.0,222520.0,37805.0,16841.0,1594.0,17585.0,46205.0,29030.0,101580.0,98801.0,120940.0,10255.0,-22214.0,5987.0,2867.0,2378.0,149970.0,0.04,1,0,0.0,4,13.854,13.854,null,null,5.166367,…,null,null,0.016196,0.016293,1.202814,1.195697,10.624314,0.413226,0,0,0,1,0,1,null,null,0.112714,613890.0275,0,100.0,null,null,null,null,null,null,null,null,null,null,null,5.0,10.0,10.0,10.0,null,null
2014-12-01,"""AAPL""",2014-09-30,2014-10-20,42123.0,24145.0,4844.0,8467.0,13134.0,23464.644,68531.0,231839.0,25077.0,27219.0,2111.0,20624.0,63448.0,28987.0,120292.0,88234.0,111547.0,13250.0,2997.0,-15380.0,2829.0,3826.0,140534.0,0.088,0,0,0.0,11,181.742,181.742,17.041999,39510.0,6.024877,…,null,null,0.016732,0.016339,0.60687,0.621467,2.962398,0.111528,0,0,0,1,0,1,1.0,null,0.172748,675019.14448,0,100.0,0.0,10.0,10.0,10.0,10.0,null,null,10.0,10.0,10.0,10.0,5.0,10.0,10.0,10.0,null,null
2015-03-01,"""AAPL""",2014-12-31,2015-01-27,74599.0,42283.0,5495.0,18024.0,26821.0,23305.676,83403.0,261894.0,32463.0,29976.0,2283.0,20392.0,73611.0,32504.0,138566.0,99141.0,123328.0,33722.0,-21165.0,-6923.0,2801.0,3217.0,155832.0,0.175,0,0,0.0,3,5.983,5.983,16.977098,44462.0,11.566302,…,0.012254,null,0.017448,0.017705,0.13023,0.128339,-14.676183,-0.529879,0,0,0,1,0,1,1.0,0.704133,0.080131,748461.823858,0,100.0,0.0,5.0,10.0,5.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,5.0,10.0,10.0,10.0,10.0,10.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2023-12-01,"""COST""",2023-08-31,2023-09-26,78939.0,69219.0,6939.0,2160.0,3681.0,442.793,35879.0,68994.0,15234.0,2285.0,16651.0,29397.0,33583.0,5377.0,43936.0,19521.0,25058.0,3725.0,-1825.0,-664.0,452.0,1556.0,31516.0,0.015,0,0,0.0,8,10.797,10.797,9.119634,6292.0,6.853662,…,0.006554,7,0.012445,0.012781,3.425729,3.335819,31.673186,

In [6]:
# Convert only integer columns to Int8, leave other types unchanged
df = df.select(
    [pl.col(col).cast(pl.Int8) if df[col].dtype.is_integer() else pl.col(col) for col in df.columns]
)
df

tic,datadate,rdq,saleq,cogsq,xsgaq,niq,ebitdaq,cshoq,actq,atq,cheq,rectq,invtq,ppentq,lctq,dlttq,ltq,req,seqq,oancfq,ivncfq,fincfq,dvq,capxq,icaptq,surprise_pct,tdq,stock_split,n_purch,val_purch,n_sales,val_sales,insider_balance,roa,roe,gpm,…,roa_yoy,roe_qoq,roe_yoy,fcf_qoq,fcf_yoy,cr_qoq,cr_yoy,qr_qoq,qr_yoy,der_qoq,der_yoy,dr_qoq,dr_yoy,ltda_yoy,pe_qoq,pe_yoy,pb_qoq,pb_yoy,ps_qoq,ps_yoy,eps_qoq,eps_yoy,ev_ebitda_qoq,ev_ebitda_yoy,ltcr_yoy,itr_yoy,rtr_yoy,atr_yoy,f_score,index_freturn,freturn,adj_freturn,adj_fperf,fperf,sector_Consumer Staples,sector_Health Care,sector_Information Technology
str,date,date,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,date,bool,i8,f64,i8,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i8,f64,f64,f64,i8,i8,i8,i8,i8
"""AAPL""",2013-12-31,2014-01-27,57594.0,33604.0,4383.0,13072.0,19607.0,24988.516,80347.0,225184.0,40711.0,25198.0,2122.0,15488.0,53769.0,16961.0,95500.0,109125.0,129684.0,22670.0,-15103.0,-7749.0,2769.0,1985.0,146645.0,0.031,2014-03-01,false,0,0.0,3,4.994,4.994,null,null,0.416536,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0.131786,0.741187,0.609401,1,1,0,0,1
"""AAPL""",2014-03-31,2014-04-23,45646.0,25812.0,4354.0,10223.0,15480.0,24128.86,70541.0,205989.0,41350.0,15820.0,1829.0,15120.0,43208.0,16962.0,85810.0,98683.0,120179.0,13538.0,11741.0,-20407.0,2661.0,1382.0,137141.0,0.14,2014-06-01,false,0,0.0,5,4.223,4.223,null,null,0.434518,…,null,null,null,-0.412328,null,0.092546,null,0.079422,null,-0.030401,null,-0.017736,null,null,null,null,0.253355,null,null,null,null,null,null,null,null,null,null,null,null,0.097818,0.468902,0.371084,1,1,0,0,1
"""AAPL""",2014-06-30,2014-07-22,37432.0,20751.0,4453.0,7748.0,12228.0,23956.684,67949.0,222520.0,37805.0,16841.0,1594.0,17585.0,46205.0,29030.0,101580.0,98801.0,120940.0,10255.0,-22214.0,5987.0,2867.0,2378.0,149970.0,0.04,2014-09-01,true,0,0.0,4,13.854,13.854,null,null,0.445635,…,null,null,null,-0.352007,null,-0.099224,null,-0.106148,null,0.176329,null,0.095835,null,null,null,null,0.118322,null,null,null,null,null,null,null,null,null,null,null,null,-0.044685,0.068892,0.113577,1,0,0,0,1
"""AAPL""",2014-09-30,2014-10-20,42123.0,24145.0,4844.0,8467.0,13134.0,23464.644,68531.0,231839.0,25077.0,27219.0,2111.0,20624.0,63448.0,28987.0,120292.0,88234.0,111547.0,13250.0,2997.0,-15380.0,2829.0,3826.0,140534.0,0.088,2014-12-01,false,0,0.0,11,181.742,181.742,0.17042,0.3542,0.426798,…,null,null,null,0.196395,null,-0.265528,null,-0.303083,null,0.283928,null,0.136609,null,null,null,null,0.192168,null,null,null,null,null,null,null,null,null,null,null,null,0.023955,0.037102,0.013147,0,0,0,0,1
"""AAPL""",2014-12-31,2015-01-27,74599.0,42283.0,5495.0,18024.0,26821.0,23305.676,83403.0,261894.0,32463.0,29976.0,2283.0,20392.0,73611.0,32504.0,138566.0,99141.0,123328.0,33722.0,-21165.0,-6923.0,2801.0,3217.0,155832.0,0.175,2015-03-01,false,0,0.0,3,5.983,5.983,0.169771,0.360518,0.433196,…,null,0.017837,null,2.236948,0.47474,0.048986,-0.24177,0.029112,-0.308009,0.041876,0.525731,0.01972,0.247571,0.647773,-0.014693,null,0.002882,0.675823,0.014431,null,0.133011,null,-0.008803,null,-0.230462,null,null,null,null,-0.059943,-0.202915,-0.142972,0,0,0,0,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""PFE""",2022-12-31,2023-01-31,24289.0,9649.0,4645.0,4994.0,5900.0,5733.0,51259.0,197205.0,22732.0,10952.0,8981.0,16274.0,42138.0,32884.0,101288.0,125656.0,95916.0,8582.0,-4410.0,-5015.0,2245.0,0.0,128645.028,0.088,2023-03-01,false,0,0.0,0,0.0,0.0,0.159083,0.327078,0.602742,…,0.313517,0.020543,0.152741,0.436558,0.451133,-0.234322,-0.130428,-0.321377,-0.198316,-0.042607,-0.213556,-0.020728,-0.103869,-0.16394,-0.251575,-0.382008,-0.236208,-0.287614,-